# 인스타그램

### 유저ID / 내용 / 작성날짜
### 결과 csv 파일로 저장

In [56]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import os
import urllib.request
import warnings # 경고메세지 제거
import requests
from datetime import datetime, timedelta
import pandas as pd

warnings.filterwarnings(action='ignore') # 경고 메세지 제거

# 로그인 계정 정보 [ID : dawaplease / PWD : dawa20152017]
ig_id = 'dawaplease'
ig_pwd = 'dawa20152017'

# 크롬드라이버 호출
driver = webdriver.Chrome("C:/project/chromedriver.exe")

# 로그인 [ID : dawaplease / PW : dawa20152017]
driver.get('https://www.instagram.com/accounts/login/?source=auth_switcher')

time.sleep(3)

id_input = driver.find_elements_by_css_selector('#react-root > section > main > div > article > div > div > div > form > div > div > label > input')[0]
id_input.send_keys(ig_id)
password_input = driver.find_elements_by_css_selector('#react-root > section > main > div > article > div > div > div > form > div > div > label > input')[1]
password_input.send_keys(ig_pwd)
password_input.submit()
time.sleep(3)

# 작품 검색
search_key = input("!주의! 검색 명내 띄어쓰기 사용 X !\n작품 검색 #")
print()
search_ig = "https://www.instagram.com/explore/tags/" + search_key + "/"

# 작품 검색 페이지 로드
driver.get(search_ig)
time.sleep(3) #웹 페이지 로드 시간 : 3초

# 마지막 결과까지 스크롤
SCROLL_PAUSE_TIME = 1.0
post_list = []

# 중복 제거
def duplicate(my_list):
    my_set = set(my_list) # 집합으로 변환
    new_list = list(my_set) # 리스트로 변환
    return new_list

# 0000년00월00일 -> 0000-00-00 "날짜 type : [datetime.date]"으로 변환
def to_datetime_type(time):
    if(time.find("분 전")>0):
        post_date = datetime.now()-timedelta(minutes=int(time.replace("분 전","")))
        return post_date.date()
    elif(time.find("시간 전")>0):
        post_date = datetime.now()-timedelta(hours=int(time.replace("시간 전","")))
        return post_date.date()
    elif(time.find("일 전")>0):
        post_date = datetime.now()-timedelta(days=int(time.replace("일 전","")))
        return post_date.date()
    else:
        post = time.split(' ')
        if(time.find("년")>0):
            post_date=datetime.now()
            post_date=post_date.replace(year=int(post[0].replace("년","")))
            post_date=post_date.replace(month=int(post[1].replace("월","")))
            post_date=post_date.replace(day=int(post[2].replace("일","")))
        else:
            post_date=datetime.now()
            post_date=post_date.replace(month=int(post[0].replace("월","")))
            post_date=post_date.replace(day=int(post[1].replace("일","")))
        return post_date.date()

# 포스트 링크 추출
while True:
    html = driver.page_source
    soup = BeautifulSoup(html, "lxml")

    for link1 in soup.find_all(name="div",attrs={"class":"Nnq7C weEfm"}):
        for num in range(3):
            try:
                title = link1.select('a')[num]
                real = title.attrs['href']
                post_list.append(real)
            except:
                break

    last_height = driver.execute_script("return document.body.scrollHeight")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:
            break
            
        else:
            last_height = new_height
            continue

post_list = duplicate(post_list) # 중복포스트 제거
post_num = len(post_list) # 검색 결과 개수
print("총 "+str(post_num)+"개의 데이터.\n")

# 포스트 데이터 추출
csvtext = []

for i in range(0, post_num):
    csvtext.append([])
    post_url = 'https://www.instagram.com'+ post_list[i]
    driver.get(post_url)
    
    # 포스트 url
    csvtext[i].append(post_url)
    
    # 포스트 작성 유저
    user_xpath = driver.find_element_by_xpath("/html/body/div[1]/section/main/div/div[1]/article/div[2]/div[1]/ul/div/li/div/div/div[2]/h2/div/a")
    user_id = user_xpath.text
    csvtext[i].append(user_id)

    # 포스트 내용
    content_xpath = driver.find_element_by_xpath("/html/body/div[1]/section/main/div/div[1]/article/div[2]/div[1]/ul/div/li/div/div/div[2]/span")
    user_content = content_xpath.text
    user_content = user_content.replace("\n","")
    csvtext[i].append(user_content)

    
    # 포스트 작성 날짜 [데이터 타입 : datetime.date]
    date_xpath = driver.find_element_by_xpath("/html/body/div[1]/section/main/div/div[1]/article/div[2]/div[2]/a/time")
    post_date = to_datetime_type(date_xpath.text)    
    csvtext[i].append(post_date)

# csv 파일로 저장
print(str(i+1)+"개의 데이터 받아오는 중.\n")
data = pd.DataFrame(csvtext, columns = ['포스트 URL', '유저ID', '내용', '작성 날짜'])
data.to_csv('ig_search.csv', encoding='utf-8', index=False)

print("저장 완료.\n")

driver.quit() # 크롬 드라이버 종료

!주의! 검색 명내 띄어쓰기 사용 X !
작품 검색 #리셋팅레이디

총 8개의 데이터.

8개의 데이터 받아오는 중.

저장 완료.



In [58]:
# 파일 확인하기
df_ig = pd.read_csv('ig_search.csv')
df_ig

,포스트 URL,유저ID,내용,작성 날짜
0,https://www.instagram.com/p/B-00H9PHaS-/,diana.pontin,리셋팅 레이디- 이셀라 에반스 낙서사실 읽은지는 좀 돼서 가물가물하기 때문에 외모묘...,2020-04-11
1,https://www.instagram.com/p/B7ivmF3Jr6a/,choinuri17,리셋팅 레이디- 이셀라 에반스 낙서사실 읽은지는 좀 돼서 가물가물하기 때문에 외모묘...,2020-01-20
2,https://www.instagram.com/p/ByxfvloFIYk/,gillllip,".「""이번에도 저와 결혼해 주시겠습니까?"".""…이번이 두번째로 최악인 청혼이예요. ...",2019-06-16
3,https://www.instagram.com/p/B8BazVend_w/,raaaheen,2020.01. [완독] 리셋팅레이디회귀물 로판. 회귀도 로판도 새롭게 해석한 소설...,2020-02-01
4,https://www.instagram.com/p/B8ekVDkHyMx/,ilikehouse1,#리셋팅레이디 #리디북스 #로맨스판타지소설 #차서진 #회귀물 #피폐물 #완결 #완결...,2020-02-13
5,https://www.instagram.com/p/Bj8_8GIHh_y/,happy_hjj,#리셋팅레이디등장인물들마저도 깨알같이 취저.시온경. 밑바닥 인생에서 돈 되는 일이면...,2018-06-13
6,https://www.instagram.com/p/CAIvqHIgSO6/,u_u9oo,"한줄 감상 : 불닭볶음면보다 매운 로맨스릴러....🪓. .『리셋팅 레이디』, 차서진...",2020-05-14
7,https://www.instagram.com/p/B73jRjhnQwX/,rooomr_0202,캐런은 117세 생일을 맞이하여 살인마가 되기로 결심했다.#리셋팅레이디,2020-01-29


# 트위터 - GetOldTweets3 사용

### 날짜 / 유저ID / 트윗 / 트윗URL

In [47]:
#GetOldTweets3 설치
!pip install GetOldTweets3

In [55]:
#import packages
import GetOldTweets3 as got
import datetime
import time
from random import uniform
from tqdm import tqdm_notebook
import pandas as pd

# 트윗 수집 기간
days_range = []

starting_date = input("트윗 수집 시작일 [ex. 양식 : 2010-07-23] : ")
ending_date = input("트윗 수집 마지막일 [ex. 양식 : 2015-07-23] : ")

start = datetime.datetime.strptime(starting_date, "%Y-%m-%d")
end = datetime.datetime.strptime(ending_date, "%Y-%m-%d")
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]

for date in date_generated:
    days_range.append(date.strftime("%Y-%m-%d"))

print("=== 설정된 트윗 수집 기간은 {} 에서 {} 까지 입니다 ===".format(days_range[0], days_range[-1]))
print("=== 총 {}일 간의 데이터 수집 ===\n".format(len(days_range)))

# 수집 기간 맞추기
start_date = days_range[0]
end_date = (datetime.datetime.strptime(days_range[-1], "%Y-%m-%d") + datetime.timedelta(days=1)).strftime("%Y-%m-%d")

# 작품 검색
search_key = input("작품 검색 #")
print()

# 트윗 수집 기준 정의
tweetCriteria = got.manager.TweetCriteria().setQuerySearch(search_key).setSince(start_date).setUntil(end_date).setMaxTweets(-1)

# 트윗 수집
print("트윗 수집 시작")
start_time = time.time()

tweet = got.manager.TweetManager.getTweets(tweetCriteria)

print("트윗 수집 완료 [{0:0.2f} Minutes]".format((time.time() - start_time)/60))
print("=== 수집 트윗 총 {}개 ===".format(len(tweet)))

# initialize
tweet_list = []

for index in tqdm_notebook(tweet):
    
    # 데이터 목록 
    username = index.username
    link = index.permalink 
    content = index.text
    tweet_date = index.date.strftime("%Y-%m-%d")
     
    info_list = [tweet_date, username, content, link]
    tweet_list.append(info_list)
    
    time.sleep(uniform(1,2))
    
tw_df = pd.DataFrame(tweet_list, columns = ["date", "user_name", "text", "link"])

# csv 파일 만들기
tw_df.to_csv("{}_tw.csv".format(search_key), index=False, encoding='utf-8')
print("저장 완료.\n")

트윗 수집 시작일 [ex. 양식 : 2010-07-23] : 2020-05-01
트윗 수집 마지막일 [ex. 양식 : 2015-07-23] : 2020-05-05
=== 설정된 트윗 수집 기간은 2020-05-01 에서 2020-05-04 까지 입니다 ===
=== 총 4일 간의 데이터 수집 ===

작품 검색 #전지적독자시점

트윗 수집 시작
트윗 수집 완료 [0.18 Minutes]
=== 수집 트윗 총 120개 ===



저장 완료.



In [59]:
# 파일 확인하기
df_tweet = pd.read_csv("전지적독자시점_tw.csv".format(search_key))
df_tweet

,date,user_name,text,link
0,2020-05-04,autumn_teatime,沈清秋你没事吧? #전지적_독자_시점 #백작가의_망나니가_되었다 #人渣反派自救系统,https://twitter.com/autumn_teatime/status/1257...
1,2020-05-04,R_LA_,[전지적 독자 시점] 한수영_리아님 @kangela1117 어린 독자_R_LA_ 갓...,https://twitter.com/R_LA_/status/1257326622727...
2,2020-05-04,ripi32478,오소마츠상 사이퍼즈 럽프듀 러브앤프로듀서 뱅드림 음양사 페이트 페그오 페스나 fat...,https://twitter.com/ripi32478/status/125732364...
3,2020-05-04,kangela1117,전지적 독자 시점 - 5부 한수영 - 리아 (@kangela1117) 어린 독자 -...,https://twitter.com/kangela1117/status/1257322...
4,2020-05-04,cooing_o91o,유중혁씨 찾아요!! 중혁이 오면 알티 이벤? 깊티 뭐라도 올릴테니 상위 트윗 알티부...,https://twitter.com/cooing_o91o/status/1257321...
...,...,...,...,...
115,2020-05-01,silversphere_,전지적 독자 시점 108화 이 부분이 앞쪽 회차들 중에선 제일 슬펐던 기억ㅠ #도결쓰다,https://twitter.com/silversphere_/status/12560...
116,2020-05-01,silversphere_,전지적 독자 시점 107화 #도결쓰다,https://twitter.com/silversphere_/status/12560...
117,2020-05-01,silversphere_,전지적 독자 시점 97화 제일 좋아하는 탑3 안에 드는 대사들이에요 특히 서울! 유...,https://twitter.com/silversphere_/status/12560...
118,2020-05-01,silversphere_,"전지적 독자 시점 76, 88화 #도결쓰다 #전독시",https://twitter.com/silversphere_/status/12560...
